# Optic Flow 

Get the optic flow from each frame for three videos

In [6]:
import sys
#sys.path.append('core')
sys.path.append('/mindhive/nklab5/users/hlk/packages/RAFT/core/')

import argparse
import os
import cv2
import glob
import numpy as np
import torch
from PIL import Image

from raft import RAFT
from utils import flow_viz
from utils.utils import InputPadder

from argparse import Namespace
from utils import flow_viz
from utils import frame_utils

DEVICE = 'cuda'

### Functions

load and view images and optic flow

In [7]:
def load_image(imfile):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(DEVICE)


def viz(img, flo):
    img = img[0].permute(1,2,0).cpu().numpy()
    flo = flo[0].permute(1,2,0).cpu().numpy()
    
    # map flow to rgb image
    flo = flow_viz.flow_to_image(flo)
    img_flo = np.concatenate([img, flo], axis=0)

    import matplotlib.pyplot as plt
    plt.imshow(img_flo / 255.0)
    #plt.imshow(flo)
    plt.axis('off')  
    plt.show()

Extract Optic flow for images using pre-trained model

In [13]:
def get_optic_flow(vDir,flowDir,mPath):
    # set the parameters for the model
    args = Namespace(alternate_corr=False, mixed_precision=False, model=mPath, path=vDir, small=False)
    model = torch.nn.DataParallel(RAFT(args))
    model.load_state_dict(torch.load(args.model))
    model = model.module
    model.to(DEVICE)
    model.eval()
    
    # use the pre-trained model
    with torch.no_grad():
        # get all of the frame names
        images = glob.glob(os.path.join(args.path, '*.png')) + \
                     glob.glob(os.path.join(args.path, '*.jpg'))
        images = sorted(images)
        frame = 1
        for imfile1, imfile2 in zip(images[:-1], images[1:]):
            image1 = load_image(imfile1)
            image2 = load_image(imfile2)
            
            # pad the images
            padder = InputPadder(image1.shape)
            image1, image2 = padder.pad(image1, image2)
            
            # extract the optic flow for each image
            flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)
        
            # save the optic flow file for each frame
            frame = frame + 1
            flo.unpad(flow_up[0]).permute(1, 2, 0).cpu().numpy()
            output_file = os.path.join(flowDir, 'of%04d.flo' % (frame))
            frame_utils.writeFlow(output_file, flow)

# Flow for Frames

Get the optic flow for each of our frames in each of our three movies and save the output

In [14]:
fDir = '/mindhive/nklab5/users/hlk/projects/vidDNN/threeVids/preProc/tripping/frames/'
oDir = '/mindhive/nklab5/users/hlk/projects/vidDNN/threeVids/preProc/tripping/opticFlow/'
mPath = '/mindhive/nklab5/users/hlk/projects/vidDNN/fove8n/learn/RAFT/models/raft-things.pth'
vids = ['tripping240','tripping339','tripping432']                        

for v in vids:
    vDir = fDir + v
    flowDir = oDir + v
    get_optic_flow(vDir,flowDir,mPath)
        